<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/ecc_lsb_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elliptic Curve Criptography

In [1]:
!pip install eciespy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00


In [2]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

In [3]:
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt

private_key = generate_eth_key()
public_key = private_key.public_key

public_key_hex = public_key.to_hex()
private_key_hex = private_key.to_hex()

print("Public Key:", public_key_hex)
print("Private Key:", private_key_hex)

ciphertext = encrypt(public_key_hex, CONTENT.encode())

print("Ciphertext:", ciphertext.hex())

decrypted_message = decrypt(private_key_hex, ciphertext)
print("Decrypted Message:", decrypted_message.decode())

Public Key: 0x2747f027860a1763c99e25927b431fc6672f93232e5d59dfb05653b82d57470e07f575f6d6b28f6cf590c9afe5b2ab845b2d06f2d791f7b10d70870eb207a29a
Private Key: 0xf5c5d58db6394ca6ce610e25348042bff099e49e748a17f5885a2723be9e4bbf
Ciphertext: 046bd0f0a8d47491fa9af19e3ba1ba28657c93d073df00d47d6e33fb00c1d7dbe9ecee7e04c795047d39a59f09dc129c9bbec7b803e07815ed676d7a17881cdbc7b1a23358fb3287d6070dbe66c108e0dbc00af9a8b4768087301cde205028678be19dc00761237dc1f3a9db59d8b78e5b76085eba9dc4619af7b09d3034a34447df659e854091e42fd860038e3f65f5d9fa35f31fa871401828324c432aa86e31c3c589ed1185241ec4eaa135db4100cc0e66d48645ec335527df929b871d44e65f315f210ca8c65bab8604786e78f7b2f3f2f3f2331c7eac79db29f79d7a724d5d4460e28e386fba12279414850b9521769bb1ca796d9a41be82c943c5d4216613e269309fe036a60190b3c033e7d27d00
Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


# Least Significant Bit

In [4]:
!git clone https://github.com/RobinDavid/LSB-Steganography && cd LSB-Steganography && ls && pip install -r requirements.txt

Cloning into 'LSB-Steganography'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 75 (delta 1), reused 2 (delta 0), pack-reused 68 (from 1)
Receiving objects: 100% (75/75), 19.72 KiB | 9.86 MiB/s, done.
Resolving deltas: 100% (26/26), done.
LICENCE  LSBSteg.py  README.md	requirements.txt
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=e5a5b03cd22fb7ef6989e4f8a701f1b90635142c2f583945bb96032528d2fab3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
#!/usr/bin/env python
# coding:UTF-8
"""LSBSteg.py

Usage:
  LSBSteg.py encode -i <input> -o <output> -f <file>
  LSBSteg.py decode -i <input> -o <output>

Options:
  -h, --help                Show this help
  --version                 Show the version
  -f,--file=<file>          File to hide
  -i,--in=<input>           Input image (carrier)
  -o,--out=<output>         Output image (or extracted file)
"""

import cv2
import docopt
import numpy as np


class SteganographyException(Exception):
    pass


class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO

            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)

    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += bytearray([int(self.read_byte(),2)])
        return output


def main():
    args = docopt.docopt(__doc__, version="0.2")
    in_f = args["--in"]
    out_f = args["--out"]
    in_img = cv2.imread(in_f)
    steg = LSBSteg(in_img)
    lossy_formats = ["jpeg", "jpg"]

    if args['encode']:
        #Handling lossy format
        out_f, out_ext = out_f.split(".")
        if out_ext in lossy_formats:
            out_f = out_f + ".png"
            print("Output file changed to ", out_f)

        data = open(args["--file"], "rb").read()
        res = steg.encode_binary(data)
        cv2.imwrite(out_f, res)

    elif args["decode"]:
        raw = steg.decode_binary()
        with open(out_f, "wb") as f:
            f.write(raw)

# if __name__=="__main__":
#     main()

# Test Image

In [7]:
import os
# Directory where the image will be saved
ori_dir_img = "ori-image"

# Create the ori_dir_img if it doesn't exist
if not os.path.exists(ori_dir_img):
    os.makedirs(ori_dir_img)

stego_dir_img = "stego-image"

# Create the stego_dir_img if it doesn't exist
if not os.path.exists(stego_dir_img):
    os.makedirs(stego_dir_img)

## Lena Image

In [8]:
import requests
from PIL import Image
import io

# URL to the raw image file
url = "https://raw.githubusercontent.com/mikolalysenko/lena/master/lena.png"

# Download the image
response = requests.get(url)
if response.status_code == 200:
    # Load the image using PIL
    lena_image = Image.open(io.BytesIO(response.content))
    lena_image.show()  # Display the image (optional)
    lena_image.save("ori-image/lena.png")  # Save the image locally
else:
    print("Failed to download the image.")

In [45]:
LENA_IMG = 'ori-image/lena.png'

if not os.path.exists(LENA_IMG):
    raise FileNotFoundError(f"Image not found at {LENA_IMG}")

# Steganography Metric

## MSE

In [46]:
def calculate_mse(original, stego):
    return np.mean((original - stego) ** 2)

## PSNR

In [60]:
def calculate_psnr(target, ori_image=LENA_IMG):
    # Read images
    original = cv2.imread(ori_image)
    stego = cv2.imread(f'{stego_dir_img}/{target}')

    if original is None:
        raise ValueError(f"Failed to load original image from {ori_image}. Ensure the file exists and is a valid image format.")
    if stego is None:
        raise ValueError(f"Failed to load stego image from {stego_dir_img}/{target}. Ensure the file exists and is a valid image format.")

    # Calculate MSE using the separated function
    mse = calculate_mse(original, stego)
    if mse == 0:  # If images are identical
        return float('inf')

    # Calculate PSNR using the 10 log10 version
    max_pixel_value = 255.0
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    # print(f'PSNR: {psnr}')
    return psnr

## SSIM

In [61]:
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(original, stego):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    stego_gray = cv2.cvtColor(stego, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM
    ssim_value, _ = ssim(original_gray, stego_gray, full=True)
    return ssim_value

## Get it together

In [66]:
def calculate_stego_metrics(target, ori_image=LENA_IMG):
    # Read images
    original = cv2.imread(ori_image)
    stego = cv2.imread(f'{stego_dir_img}/{target}')

    if original is None:
        raise ValueError(f"Failed to load original image from {ori_image}. Ensure the file exists and is a valid image format.")
    if stego is None:
        raise ValueError(f"Failed to load stego image from {stego_dir_img}/{target}. Ensure the file exists and is a valid image format.")

    # Calculate metrics
    mse_value = calculate_mse(original, stego)
    psnr_value = calculate_psnr(target)
    ssim_value = calculate_ssim(original, stego)

    # Print results
    print(f'Metrics between original ({ori_image}) and stego image ({target}):')
    print(f'MSE: {mse_value}')
    print(f'PSNR: {psnr_value}')
    print(f'SSIM: {ssim_value}')

    return mse_value, psnr_value, ssim_value

# LSB Helper

In [50]:
def embed_lsb(target, content, ori_img = LENA_IMG):
  steg = LSBSteg(cv2.imread(ori_img))

  img_encoded = steg.encode_text(content)

  lsb_img = f"{stego_dir_img}/{target}"

  cv2.imwrite(lsb_img, img_encoded)

In [51]:
def extract_lsb(target):
  im = cv2.imread(f"{stego_dir_img}/{target}")

  steg = LSBSteg(im)

  content = steg.decode_text()

  return content

# LSB Only

In [52]:
embed_lsb("lsb_only.png", CONTENT)

In [53]:
extract_lsb("lsb_only.png")

'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

In [54]:
extract_lsb("lsb_only.png") == CONTENT

True

In [67]:
calculate_stego_metrics("lsb_only.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_only.png):
MSE: 0.0009028116861979166
PSNR: 78.57483188820159
SSIM: 0.9999987952461553


(0.0009028116861979166, 78.57483188820159, 0.9999987952461553)

# LSB with ECC

In [25]:
ciphertext

b"\x04k\xd0\xf0\xa8\xd4t\x91\xfa\x9a\xf1\x9e;\xa1\xba(e|\x93\xd0s\xdf\x00\xd4}n3\xfb\x00\xc1\xd7\xdb\xe9\xec\xee~\x04\xc7\x95\x04}9\xa5\x9f\t\xdc\x12\x9c\x9b\xbe\xc7\xb8\x03\xe0x\x15\xedgmz\x17\x88\x1c\xdb\xc7\xb1\xa23X\xfb2\x87\xd6\x07\r\xbef\xc1\x08\xe0\xdb\xc0\n\xf9\xa8\xb4v\x80\x870\x1c\xde P(g\x8b\xe1\x9d\xc0\x07a#}\xc1\xf3\xa9\xdbY\xd8\xb7\x8e[v\x08^\xba\x9d\xc4a\x9a\xf7\xb0\x9d04\xa3DG\xdfe\x9e\x85@\x91\xe4/\xd8`\x03\x8e?e\xf5\xd9\xfa5\xf3\x1f\xa8q@\x18(2LC*\xa8n1\xc3\xc5\x89\xed\x11\x85$\x1e\xc4\xea\xa15\xdbA\x00\xcc\x0ef\xd4\x86E\xec3U'\xdf\x92\x9b\x87\x1dD\xe6_1_!\x0c\xa8\xc6[\xab\x86\x04xnx\xf7\xb2\xf3\xf2\xf3\xf23\x1c~\xacy\xdb)\xf7\x9dzrM]D`\xe2\x8e8o\xba\x12'\x94\x14\x85\x0b\x95!v\x9b\xb1\xcaym\x9aA\xbe\x82\xc9C\xc5\xd4!f\x13\xe2i0\x9f\xe06\xa6\x01\x90\xb3\xc03\xe7\xd2}\x00"

In [26]:
embed_lsb("lsb_w_ECC.png", ciphertext.hex())

In [27]:
extracted_data = extract_lsb("lsb_w_ECC.png")
extracted_data

'046bd0f0a8d47491fa9af19e3ba1ba28657c93d073df00d47d6e33fb00c1d7dbe9ecee7e04c795047d39a59f09dc129c9bbec7b803e07815ed676d7a17881cdbc7b1a23358fb3287d6070dbe66c108e0dbc00af9a8b4768087301cde205028678be19dc00761237dc1f3a9db59d8b78e5b76085eba9dc4619af7b09d3034a34447df659e854091e42fd860038e3f65f5d9fa35f31fa871401828324c432aa86e31c3c589ed1185241ec4eaa135db4100cc0e66d48645ec335527df929b871d44e65f315f210ca8c65bab8604786e78f7b2f3f2f3f2331c7eac79db29f79d7a724d5d4460e28e386fba12279414850b9521769bb1ca796d9a41be82c943c5d4216613e269309fe036a60190b3c033e7d27d00'

In [68]:
calculate_stego_metrics("lsb_w_ECC.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_w_ECC.png):
MSE: 0.0028025309244791665
PSNR: 73.65529947359488
SSIM: 0.9999928013838967


(0.0028025309244791665, 73.65529947359488, 0.9999928013838967)

In [28]:
extracted_data == ciphertext.hex()

True

In [30]:
extracted_data = bytes.fromhex(extracted_data)
extracted_data

b"\x04k\xd0\xf0\xa8\xd4t\x91\xfa\x9a\xf1\x9e;\xa1\xba(e|\x93\xd0s\xdf\x00\xd4}n3\xfb\x00\xc1\xd7\xdb\xe9\xec\xee~\x04\xc7\x95\x04}9\xa5\x9f\t\xdc\x12\x9c\x9b\xbe\xc7\xb8\x03\xe0x\x15\xedgmz\x17\x88\x1c\xdb\xc7\xb1\xa23X\xfb2\x87\xd6\x07\r\xbef\xc1\x08\xe0\xdb\xc0\n\xf9\xa8\xb4v\x80\x870\x1c\xde P(g\x8b\xe1\x9d\xc0\x07a#}\xc1\xf3\xa9\xdbY\xd8\xb7\x8e[v\x08^\xba\x9d\xc4a\x9a\xf7\xb0\x9d04\xa3DG\xdfe\x9e\x85@\x91\xe4/\xd8`\x03\x8e?e\xf5\xd9\xfa5\xf3\x1f\xa8q@\x18(2LC*\xa8n1\xc3\xc5\x89\xed\x11\x85$\x1e\xc4\xea\xa15\xdbA\x00\xcc\x0ef\xd4\x86E\xec3U'\xdf\x92\x9b\x87\x1dD\xe6_1_!\x0c\xa8\xc6[\xab\x86\x04xnx\xf7\xb2\xf3\xf2\xf3\xf23\x1c~\xacy\xdb)\xf7\x9dzrM]D`\xe2\x8e8o\xba\x12'\x94\x14\x85\x0b\x95!v\x9b\xb1\xcaym\x9aA\xbe\x82\xc9C\xc5\xd4!f\x13\xe2i0\x9f\xe06\xa6\x01\x90\xb3\xc03\xe7\xd2}\x00"

In [31]:
decrypted_message = decrypt(private_key_hex, extracted_data)
print("Decrypted Message:", decrypted_message.decode())

Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [32]:
decrypted_message.decode() == CONTENT

True